# Basic Search

The CDA provides a custom python tool for searching CDA data. [`Q`](usage/#q) (short for Query) offers several ways to search and filter data, and several input modes:

---
- **<a href="../../QuickStart/usage/#q">Q.()</a>** builds a query that can be used by `run()` or `count()`
- **<a href="../../QuickStart/usage/#qrun">Q.run()</a>** returns data for the specified search 
- **<a href="../../QuickStart/usage/#qcount">Q.count()</a>** returns summary information (counts) data that fit the specified search
- **<a href="../../QuickStart/usage/#columns">columns()</a>** returns entity field names
- **<a href="../../QuickStart/usage/#unique_terms">unique_terms()</a>** returns entity field contents

---
                                                                    
Before we do any work, we need to import these functions from cdapython. We're also importing functions from several other packages to make viewing and manipulating tables easier. The `opt.` settings are pre-configuring how itables should display our tables, with scrolling and paging enabled.
Finally, we're telling cdapython to report it's version so we can be sure we're using the one we mean to:

In [1]:
from cdapython import Q, columns, unique_terms, query
import numpy as np
import pandas as pd
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=True)
import itables.options as opt
opt.maxBytes=0
opt.scrollX="200px"
opt.scrollCollapse=True
opt.paging=True
opt.maxColumns=0
print(Q.get_version())

<IPython.core.display.Javascript object>

2022.10.24

In [2]:
# Delete me
Q.set_default_project_dataset("gdc-bq-sample.dev")
Q.set_table_version("all_Subjects_v3_1_test1")
Q.set_host_url("http://35.192.60.10:8080/")
Q.get_host_url()
Q.get_default_project_dataset()

'gdc-bq-sample.dev'

<div class="cdanote" style="background-color:#b3e5d5;color:black;padding:20px;">
    
CDA data comes from three sources:
<ul>
<li><b>The <a href="https://proteomic.datacommons.cancer.gov/pdc/"> Proteomic Data Commons</a> (PDC)</b></li>
<li><b>The <a href="https://gdc.cancer.gov/">Genomic Data Commons</a> (GDC)</b></li>
<li><b>The <a href="https://datacommons.cancer.gov/repository/imaging-data-commons">Imaging Data Commons</a> (IDC)</b></li>
</ul> 
    
The CDA makes this data searchable in four main endpoints:

<ul>
<li><b>subject:</b> A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</li>
<li><b>researchsubject:</b> A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. A subject who participates in 3 studies will have 3 researchsubject IDs.</li>
<li><b>specimen:</b> Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</li>
<li><b>mutation:</b> Molecular data about specific mutations, currently limited to the TCGA-READ project from GDC.</li>
</ul>
one endpoint that can be added to the above to get the relevant files:    
<ul>
<li><b>file:</b> A unit of data about subjects, researchsubjects, specimens, or their associated information.</li>
</ul>
and two endpoints that offer deeper information about data in the researchsubject endpoint:
<ul>
<li><b>diagnosis:</b> A collection of characteristics that describe an abnormal condition of the body as assessed at a point in time. May be used to capture information about neoplastic and non-neoplastic conditions.</li>
<li><b>treatment:</b> Represent medication administration or other treatment types.</li>
</ul>
</div>


If you are looking to build a cohort of distinct individuals who meet some criteria, search by `subject`. If you want to build a cohort, but are particularly interested in studies rather than the participates per se, search by `researchsubject`. If you are looking for biosamples that can be ordered or a specific format of information (for e.g. histological slides) start with `specimen`. If you are primarily looking for files you can reuse for your own analysis, add `.file` to your call.

In the four 'main' tables, all of the rows will have one or more files associated with them that can be directly found by chaining, as in `specimen.files`. Diagnosis and treatment do not have files directly associated with them, so a query statemet of `diagnosis.files` or `treatment.file` will not work.

Any metadata field can be searched from any endpoint, the only difference between search types is what type of data is returned by default. This means that you can think of the CDA as a really, really enormous spreadsheet full of data. To search this enormous spreadsheet, you'd want select columns, and then filter rows.



## Basic search with endpoints

Let's try a broad search of the CDA to see what information exists about cancers that were first diagnosed in the brain. To run this simple search, we would first construct a query in `Q` where the syntax is `Q('<name_of_column> = "<unique_term>"'` and save it to a variable `myquery`:

In [3]:
myquery = Q('primary_diagnosis_site = "brain"')


<div class="cdawarn" style="background-color:#f9cfbf;color:black;padding:20px;">
    
<h3>Where did those terms come from?</h3>
    
If you aren't sure how we knew what terms to put in our search, please refer back to the <a href="../SearchTerms">What search terms are available?</a> notebook. 
</div>

### subject
Now we can use that query to search any of information types. Let's start by looking at what subjects meet our criteria. To do that, we will send our query to the subject endpoint, then ask for it to run:

In [4]:
subjectresults = myquery.subject.run()

Getting results from database

Total execution time: 0
                        min 4.106 sec 4106 ms

We saved the output in a variable `subjectresults`, so we don't get much visible output. To see what our results are, we need to look into the variable. The simplest way is to call `subjectresults` directly:

In [5]:
subjectresults


            
            Offset: 0
            Count: 100
            Total Row Count: 3015
            More pages: True
            

This tells us four parameters that describe our results:

---

- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows are in the full results table
- **More pages:** This is always a True or False. False means that our current page has all the available results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---

In any results variable, you can also use `print()` to display a 'pretty print' version of the results. This color codes the display in a way that can make it easier to find the information you care about:


In [6]:
print(subjectresults)

Offset: 0
            Count: 100
            Total Row Count: 3015
            More pages: True

We have 2384 subject matches, let's look at the actual table. The easiest way to do this is by using the python function `.to_dataframe()` on our `subjectresults` variable:

In [7]:
subjectresults.to_dataframe()

subject_id  \
0       ACRIN-DSC-MR-Brain-038   
1                      C115251   
2                    C3L-02970   
3                    C3L-03549   
4                    C3L-03744   
..                         ...   
95  RIDER Neuro MRI-1023805636   
96  RIDER Neuro MRI-5244517593   
97                TCGA-02-0074   
98                TCGA-02-0113   
99                TCGA-02-0266   

                                   subject_identifier       species     sex  \
0   [{'system': 'IDC', 'value': 'ACRIN-DSC-MR-Brai...  homo sapiens    None   
1             [{'system': 'PDC', 'value': 'C115251'}]  homo sapiens    male   
2   [{'system': 'GDC', 'value': 'C3L-02970'}, {'sy...  homo sapiens    male   
3           [{'system': 'IDC', 'value': 'C3L-03549'}]  homo sapiens    None   
4   [{'system': 'GDC', 'value': 'C3L-03744'}, {'sy...  homo sapiens    male   
..                                                ...           ...     ...   
95  [{'system': 'IDC', 'value': 'RIDER Neuro MRI-1...  homo sapiens    None   
96  [{'system': 'IDC', 'value': 'RIDER Neuro MRI-5...  homo sapiens    None   
97  [{'system': 'GDC', 'value': 'TCGA-02-0074'}, {...  homo sapiens  female   
98  [{'system': 'GDC', 'value': 'TCGA-02-0113'}, {...  homo sapiens  female   
99  [{'system': 'GDC', 'value': 'TCGA-02-0266'}, {...  homo sapiens    male   

            race               ethnicity  days_to_birth  \
0           None                    None            NaN   
1   not reported      hispanic or latino            NaN   
2          white  not hispanic or latino       -21524.0   
3           None                    None            NaN   
4          white  not hispanic or latino       -27552.0   
..           ...                     ...            ...   
95          None                    None            NaN   
96          None                    None            NaN   
97         white  not hispanic or latino            NaN   
98         white  not hispanic or latino            NaN   
99  not reported      hispanic or latino            NaN   

                           subject_associated_project vital_status  \
0                                [acrin_dsc_mr_brain]         None   
1   [Proteogenomic Analysis of Pediatric Brain Can...        Alive   
2              [cptac_gbm, CPTAC-3, CPTAC3-Discovery]         Dead   
3                                         [cptac_gbm]         None   
4              [cptac_gbm, CPTAC-3, CPTAC3-Discovery]        Alive   
..                                                ...          ...   
95                                  [rider_neuro_mri]         None   
96                                  [rider_neuro_mri]         None   
97                               [TCGA-GBM, tcga_gbm]         Dead   
98                               [TCGA-GBM, tcga_gbm]        Alive   
99                               [TCGA-GBM, tcga_gbm]         Dead   

    days_to_death cause_of_death  
0             NaN           None  
1             NaN   Not Reported  
2           381.0           None  
3             NaN           None  
4             NaN           None  
..            ...            ...  
95            NaN           None  
96            NaN           None  
97            NaN           None  
98            NaN           None  
99            NaN           None  

[100 rows x 11 columns]

By default `to_dataframe()` shows us the first and last five rows for the first page of our results, so we can easily preview our data.

Since we queried the Subject endpoint, our default results tell us Subject level information, that is, information about unique individuals: their sex, race, age, species, etc. The `id` column tells us the unique identifier for each individual. The identifier column has nested information about what study or studies a Subject participated in, and will list all of their researchsubject identifiers. 

The `to_dataframe()` function converts the results to a pandas dataframe. So if we save the dataframe to a variable, we can use any [pandas dataframe functions](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) to explore it. For example, lets see whether any of the Subjects in our first 100 results are black or african american. First we'll save the results to a dataframe, then subset that dataframe to only show rows where the word "black" appears in the "race" column. "NAs" which are shown as "None" in these tables, so for our filter to work, we'll need to specifically tell it to ignore NAs:


In [8]:
subjectdata = subjectresults.to_dataframe()
subjectdata[subjectdata['race'].str.contains("black", na=False)]

subject_id                                 subject_identifier  \
63  TCGA-02-2485  [{'system': 'GDC', 'value': 'TCGA-02-2485'}, {...   
70  TCGA-19-A6J4  [{'system': 'GDC', 'value': 'TCGA-19-A6J4'}, {...   
73  TCGA-E1-A7YS  [{'system': 'GDC', 'value': 'TCGA-E1-A7YS'}, {...   

         species   sex                       race               ethnicity  \
63  homo sapiens  male  black or african american  not hispanic or latino   
70  homo sapiens  male  black or african american  not hispanic or latino   
73  homo sapiens  male  black or african american  not hispanic or latino   

    days_to_birth subject_associated_project vital_status  days_to_death  \
63            NaN       [TCGA-GBM, tcga_gbm]        Alive            NaN   
70            NaN       [TCGA-GBM, tcga_gbm]         Dead            NaN   
73            NaN       [tcga_lgg, TCGA-LGG]         Dead            NaN   

   cause_of_death  
63           None  
70           None  
73           None

There are subjects in our first hundred results that meet the criteria. If we just want to be sure that the data contains some value, this might be good enough. But often we want to search the entire set of results and not just the first page. 

We'll cover how to work with large results dataframes in the <a href="../Pagination">Pagination</a> notebook. Or, learn how to get summary information from search results in the <a href="../DataSummaries">Data Summaries</a> notebook.

---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Subject Field Definitions</h3>

<i>A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</i>

    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system."</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>species:</b> The taxonomic group (e.g. species) of the subject.</li>
<li><b>sex:</b> The biologic character or quality that distinguishes male and female from one another as expressed by analysis of the person's gonadal, morphologic (internal and external), chromosomal, and hormonal characteristics.</li>
<li><b>race:</b> An arbitrary classification of a taxonomic group that is a division of a species. It usually arises as a consequence of geographical isolation within a species and is characterized by shared heredity, physical attributes and behavior, and in the case of humans, by common history, nationality, or geographic distribution. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>ethnicity:</b> An individual's self-described social and cultural grouping, specifically whether an individual describes themselves as Hispanic or Latino. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>days_to_birth:</b> Number of days between the date used for index and the date from a person's date of birth represented as a calculated negative number of days.</li>
<li><b>subject_associated_project:</b> The list of Projects associated with the Subject.</li>
<li><b>vital_status:</b> Coded value indicating the state or condition of being living or deceased; also includes the case where the vital status is unknown.</li>
<li><b>days_to_death:</b> Number of days between the date used for index and the date from a person's date of death represented as a calculated number of days.</li>
<li><b>cause_of_death:</b> Coded value indicating the circumstance or condition that results in the death of the subject.</li>
</ul>  

</div>
    
---

### researchsubject

If we're interested in what researchsubjects meet our criteria, we can run our query against the researchsubject endpoint:

In [9]:
researchsubjectresults = myquery.researchsubject.run()
print(researchsubjectresults)

Getting results from database

Total execution time: 0
                        min 4.036 sec 4036 ms

Offset: 0
            Count: 100
            Total Row Count: 4347
            More pages: True

Now we see that our 2384 subjects have 3716 researchsubjects between them, that means that some, but not all, of our subjects were participants in more than one study. Let's peek at the data:

In [10]:
researchsubjectresults.to_dataframe()

researchsubject_id  \
0   RIDER Neuro MRI-5244517593__rider_neuro_mri   
1          cde1d14f-d067-4419-aa03-eb785dec8235   
2             PGBM-010__brain_tumor_progression   
3   RIDER Neuro MRI-1086100996__rider_neuro_mri   
4          54ada9c7-e8f7-4ecc-b829-903be7e3ad82   
..                                          ...   
95                                   W9__ivygap   
96                                  W50__ivygap   
97                                  W33__ivygap   
98                                  W10__ivygap   
99                                  W39__ivygap   

                           researchsubject_identifier  \
0   [{'system': 'IDC', 'value': 'RIDER Neuro MRI-5...   
1   [{'system': 'GDC', 'value': 'cde1d14f-d067-441...   
2   [{'system': 'IDC', 'value': 'PGBM-010__brain_t...   
3   [{'system': 'IDC', 'value': 'RIDER Neuro MRI-1...   
4   [{'system': 'GDC', 'value': '54ada9c7-e8f7-4ec...   
..                                                ...   
95         [{'system': 'IDC', 'value': 'W9__ivygap'}]   
96        [{'system': 'IDC', 'value': 'W50__ivygap'}]   
97        [{'system': 'IDC', 'value': 'W33__ivygap'}]   
98        [{'system': 'IDC', 'value': 'W10__ivygap'}]   
99        [{'system': 'IDC', 'value': 'W39__ivygap'}]   

   member_of_research_project primary_diagnosis_condition  \
0             rider_neuro_mri                        None   
1                   GENIE-JHU              Neoplasms, NOS   
2     brain_tumor_progression                        None   
3             rider_neuro_mri                        None   
4                   GENIE-NKI              Neoplasms, NOS   
..                        ...                         ...   
95                     ivygap                        None   
96                     ivygap                        None   
97                     ivygap                        None   
98                     ivygap                        None   
99                     ivygap                        None   

   primary_diagnosis_site                  subject_id  
0                   Brain  RIDER Neuro MRI-5244517593  
1                   Brain             GENIE-JHU-03818  
2                   Brain                    PGBM-010  
3                   Brain  RIDER Neuro MRI-1086100996  
4                   Brain              GENIE-NKI-8IN3  
..                    ...                         ...  
95                  Brain                          W9  
96                  Brain                         W50  
97                  Brain                         W33  
98                  Brain                         W10  
99                  Brain                         W39  

[100 rows x 6 columns]

Each row from the researchsubject endpoint results tells us about a subject in a given study. Using this endpoint we can find out information like what studies fit our search criteria, and also get data that we can filter to have only subjects from multiple studies, or only subjects from single studies.

Any given subject will have one row per study they participated in. The subject_id in the last column of this view is the same as the `id` in the first column of the Subjects endpoint results. You can use this to combine information across endpoints, which is covered near the end of the <a href="../BuildingACohort/#merging-results-across-endpoints">Cohort Building workflow</a> notebook.

---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>ResearchSubject Field Definitions</h3>

<i>A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. A subject who participates in 3 studies will have 3 researchsubject IDs</i>
    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. For CDA, this is case_id.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>member_of_research_project:</b> A reference to the Project(s) of which this ResearchSubject is a member.</li>
<li><b>primary_diagnosis_condition:</b> The text term used to describe the type of malignant disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This attribute represents the disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>primary_diagnosis_site:</b> The text term used to describe the primary site of disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This categorization groups cases into general categories. This attribute represents the primary site of disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>subject_id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. Can be joined to the `id` field from subject results</li>
</ul>  

</div>
    
---

### diagnosis

The diagnosis endpoint is an extension of the researchsubject endpoint, and returns information about researchsubjects that have a diagnosis that meets our search criteria:

In [11]:
diagnosisresults = myquery.diagnosis.run()
diagnosisresults.to_dataframe()

Getting results from database

Total execution time: 0
                        min 4.228 sec 4228 ms

diagnosis_id  \
0   02ce9b1f-18f4-45ec-a487-63f9c74bb422   
1   0f7bb10c-a17e-4a98-b559-461edbf840c8   
2   13c464b8-bd62-5d47-91ff-4b2c9183c48b   
3   195c5853-a955-551b-8601-6f0d8a08cea8   
4   2faa8b42-f38e-50f2-9920-cafeafc989ae   
..                                   ...   
95  bff99495-d30b-5f3f-87f1-c50ed19e8204   
96  c253a6c1-493e-5c37-9256-a13080252bec   
97  c37fa63b-6757-5a5d-afbf-ba3f78a9f93b   
98  d0929138-ff5e-11e9-9a07-0a80fada099c   
99  d092cf33-ff5e-11e9-9a07-0a80fada099c   

                                 diagnosis_identifier       primary_diagnosis  \
0   [{'system': 'GDC', 'value': '02ce9b1f-18f4-45e...   Mixed germ cell tumor   
1   [{'system': 'GDC', 'value': '0f7bb10c-a17e-4a9...            Glioblastoma   
2   [{'system': 'GDC', 'value': '13c464b8-bd62-5d4...  Oligodendroglioma, NOS   
3   [{'system': 'GDC', 'value': '195c5853-a955-551...            Glioblastoma   
4   [{'system': 'GDC', 'value': '2faa8b42-f38e-50f...            Glioblastoma   
..                                                ...                     ...   
95  [{'system': 'GDC', 'value': 'bff99495-d30b-5f3...            Glioblastoma   
96  [{'system': 'GDC', 'value': 'c253a6c1-493e-5c3...            Glioblastoma   
97  [{'system': 'GDC', 'value': 'c37fa63b-6757-5a5...            Glioblastoma   
98  [{'system': 'PDC', 'value': 'd0929138-ff5e-11e...    Medulloblastoma, NOS   
99  [{'system': 'PDC', 'value': 'd092cf33-ff5e-11e...       Glioma, malignant   

    age_at_diagnosis morphology    stage         grade method_of_diagnosis  \
0                NaN     9085/3     None  Not Reported                None   
1            23121.0     9440/3     None  Not Reported                None   
2            12764.0     9450/3     None  not reported                None   
3            14838.0     9440/3     None  not reported                None   
4            28696.0     9440/3     None  not reported                None   
..               ...        ...      ...           ...                 ...   
95           20355.0     9440/3     None  not reported                None   
96           28496.0     9440/3     None  not reported                None   
97           10699.0     9440/3     None  not reported                None   
98            3557.0     9470/3  Unknown            G4        Not Reported   
99            2446.0     9380/3  Unknown    High Grade        Not Reported   

             subject_id                    researchsubject_id  
0   GENIE-MSK-P-0002849  3ecb27b2-75cc-44d0-879e-035e19f39193  
1             C3N-01366  a1da2f64-5a24-4e6b-abc1-e70aae23082c  
2          TCGA-DU-5870  eb42625e-fbbb-4bee-bbe9-315b3ca2138a  
3          TCGA-08-0351  91063e7e-60db-47a5-aeca-af7c37b1074e  
4          TCGA-02-0281  42fbe95f-9068-45a8-a8a2-c24db757eae6  
..                  ...                                   ...  
95         TCGA-15-1446  05d5da50-52fe-4342-9c15-9ee4c1d6662e  
96         TCGA-26-5136  34f216fb-09dc-48af-9e09-7d12cc07c1f7  
97         TCGA-02-0104  db38e6b7-102f-4670-875f-b327790c70e4  
98               C72570  d08ddc45-ff5e-11e9-9a07-0a80fada099c  
99              C952635  d08e0ac0-ff5e-11e9-9a07-0a80fada099c  

[100 rows x 10 columns]


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Diagnosis Field Definitions</h3>

<i>A collection of characteristics that describe an abnormal condition of the body as assessed at a point in time. May be used to capture information about neoplastic and non-neoplastic conditions.</i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>primary_diagnosis:</b> The diagnosis instance that qualified a subject for inclusion on a ResearchProject.</li>
  <li><b>age_at_diagnosis:</b> The age in days of the individual at the time of diagnosis.</li>
  <li><b>morphology:</b> Code that represents the histology of the disease using the third edition of the <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a>, published in 2000, used principally in tumor and cancer registries for coding the site (topography) and the histology (morphology) of neoplasms.</li>
  <li><b>stage:</b> The extent of a cancer in the body. Staging is usually based on the size of the tumor, whether lymph nodes contain cancer, and whether the cancer has spread from the original site to other parts of the body. Different diseases may use different staging criteria, please refer to the originating data source to see what staging system is reported</li>
  <li><b>grade:</b> The degree of abnormality of cancer cells, a measure of differentiation, the extent to which cancer cells are similar in appearance and function to healthy cells of the same tissue type. The degree of differentiation often relates to the clinical behavior of the particular tumor. Based on the microscopic findings, tumor grade is commonly described by one of four degrees of severity. Histopathologic grade of a tumor may be used to plan treatment and estimate the future course, outcome, and overall prognosis of disease. Certain types of cancers, such as soft tissue sarcoma, primary brain tumors, lymphomas, and breast have special grading systems.</li>
  <li><b>method_of_diagnosis:</b> The method used to confirm the patients malignant diagnosis.</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---




### treatment

The treatment endpoint is an extension of diagnosis and returns information about treatments undertaken on researchsubjects that have a given diagnosis that meets our search criteria:

In [12]:
treatmentresults = myquery.treatment.run()
treatmentresults.to_dataframe()

Getting results from database

Total execution time: 0
                        min 4.308 sec 4308 ms

treatment_id  \
0   1e55709a-abb4-508b-bff6-c9b7ecc9e382   
1   2aab0550-36ff-5904-908a-34477a7cd5e0   
2   307c2986-043e-504d-b072-55a0e37a9651   
3   36f79c38-4002-519d-94d4-86565e475221   
4   3ee29e84-92e9-581d-9afd-bbc8ceede124   
..                                   ...   
95  8aafe70f-0f32-5456-ad79-7ba416d5b966   
96  90884312-5ab6-5d2f-bc87-bba86a61f545   
97  9b35c886-b6f7-5ddb-adbb-45aff90087ba   
98  bcd570a3-712a-575f-8bda-4e573ef28179   
99  c657d1f1-6f3f-5e64-86ab-3e4482e5fc44   

                                 treatment_identifier treatment_type  \
0   [{'system': 'GDC', 'value': '1e55709a-abb4-508...           None   
1   [{'system': 'GDC', 'value': '2aab0550-36ff-590...           None   
2   [{'system': 'GDC', 'value': '307c2986-043e-504...           None   
3   [{'system': 'GDC', 'value': '36f79c38-4002-519...           None   
4   [{'system': 'GDC', 'value': '3ee29e84-92e9-581...           None   
..                                                ...            ...   
95  [{'system': 'GDC', 'value': '8aafe70f-0f32-545...           None   
96  [{'system': 'GDC', 'value': '90884312-5ab6-5d2...           None   
97  [{'system': 'GDC', 'value': '9b35c886-b6f7-5dd...           None   
98  [{'system': 'GDC', 'value': 'bcd570a3-712a-575...           None   
99  [{'system': 'GDC', 'value': 'c657d1f1-6f3f-5e6...           None   

   treatment_outcome days_to_treatment_start days_to_treatment_end  \
0               None                    None                  None   
1               None                    None                  None   
2               None                    None                  None   
3               None                    None                  None   
4               None                    None                  None   
..               ...                     ...                   ...   
95              None                    None                  None   
96              None                    None                  None   
97              None                    None                  None   
98              None                    None                  None   
99              None                    None                  None   

   therapeutic_agent treatment_anatomic_site treatment_effect  \
0               None                    None             None   
1               None                    None             None   
2               None                    None             None   
3               None                    None             None   
4               None                    None             None   
..               ...                     ...              ...   
95              None                    None             None   
96              None                    None             None   
97              None                    None             None   
98              None                    None             None   
99              None                    None             None   

   treatment_end_reason number_of_cycles    subject_id  \
0                  None             None  TCGA-DB-A4XB   
1                  None             None  TCGA-DH-A66F   
2                  None             None  TCGA-S9-A6WM   
3                  None             None  TCGA-02-0024   
4                  None             None  TCGA-HT-7604   
..                  ...              ...           ...   
95                 None             None  TCGA-FG-6688   
96                 None             None  TCGA-06-0397   
97                 None             None  TCGA-HT-7608   
98                 None             None  TCGA-HT-7860   
99                 None             None  TCGA-06-2557   

                      researchsubject_id                          diagnosis_id  
0   fb6437f9-ade8-4c33-a0d0-ceacb2bf784d  c4ba02cf-341d-51e8-8a1a-a6aeccea4397  
1   cfcd3835-bb09-400e-b7eb-55ac139d01f8  1d497e09-8d45-557e-abaa-3f1f0582b6a5  
2   f79420ca-b0ae-417c-95f7-d28c35760202  c7a7a135-a3c0-52eb-8d39-cf7fa5


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Treatment Field Definitions</h3>

<i><i> Medication administration or other treatment types. A single research subject may have multiple treatments for a single diagnosis, and/or different diagnoses, and different treatments, across different studies</i></i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>treatment_type:</b> The treatment type including medication/therapeutics or other procedures.</li>
  <li><b>treatment_outcome:</b> The final outcome of the treatment.</li>
  <li><b>days_to_treatment_start:</b> The timepoint at which the treatment started.</li>
  <li><b>days_to_treatment_end:</b>The timepoint at which the treatment ended. </li>
  <li><b>therapeutic_agent:</b> One or more therapeutic agents as part of this treatment.</li>
  <li><b>treatment_anatomic_site:</b> The anatomical site that the treatment targets.</li>
  <li><b>treatment_effect:</b>The effect of a treatment on the diagnosis or tumor. </li>
  <li><b>treatment_end_reason:</b>The reason the treatment ended. </li>
  <li><b>number_of_cycles:</b>The number of treatment cycles the subject received. </li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
  <li><b>researchsubject_diagnosis_id:</b> An identifier for the diagnosis. Can be joined to the `id` field from diagnosis results</li>
</ul>  

</div>
    
---




### specimen

We can use this same query to see what specimens are available for brain tissue at the CDA:

In [13]:
specimenresults =  myquery.specimen.run()
specimenresults

Getting results from database

Total execution time: 0
                        min 4.179 sec 4179 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 39220
            More pages: True
            

Nearly 40,000 specimens meet our search criteria! We would typically expect this number to be much larger than our number of subjects or researchsubjects. First because studies will often take more than one sample per subject, and second because any given specimen might be aliquoted out to be used in multiple tests. Since we didn't specify any further filters, our results will return all of these as separate speciments. Let's look at a few:

In [14]:
specimenresults.to_dataframe()

specimen_id  \
0   0024bf91-4ba7-4e0d-9e23-cb37188324c7   
1   01b98b0e-bbfc-4e0f-934a-3a6ac9486342   
2   01ec04df-91c8-4c46-a71d-fe1abe06a221   
3   0242c021-1602-420e-a245-8ee1052fdcef   
4   031453eb-67ac-416e-8350-2e35f55c7773   
..                                   ...   
95  3abeff8e-5bc2-4cb9-aa06-55bd17d41f82   
96  3b60125e-2eed-4a75-9811-b7749dbe7977   
97  3b9c4291-4b37-413e-877e-0b557e960c64   
98  3bc3a6ac-de5e-4a7b-8fea-a9cf6f4bfd18   
99  3c83ded3-3515-4071-8788-6e8bf2c2a173   

                                  specimen_identifier  \
0   [{'system': 'GDC', 'value': '0024bf91-4ba7-4e0...   
1   [{'system': 'GDC', 'value': '01b98b0e-bbfc-4e0...   
2   [{'system': 'GDC', 'value': '01ec04df-91c8-4c4...   
3   [{'system': 'GDC', 'value': '0242c021-1602-420...   
4   [{'system': 'GDC', 'value': '031453eb-67ac-416...   
..                                                ...   
95  [{'system': 'GDC', 'value': '3abeff8e-5bc2-4cb...   
96  [{'system': 'GDC', 'value': '3b60125e-2eed-4a7...   
97  [{'system': 'GDC', 'value': '3b9c4291-4b37-413...   
98  [{'system': 'GDC', 'value': '3bc3a6ac-de5e-4a7...   
99  [{'system': 'GDC', 'value': '3c83ded3-3515-407...   

   specimen_associated_project  days_to_collection primary_disease_type  \
0                     TCGA-LGG                78.0              Gliomas   
1                     TCGA-LGG                 NaN              Gliomas   
2                     TCGA-LGG               219.0              Gliomas   
3                     TCGA-LGG                 0.0              Gliomas   
4                     TCGA-GBM                 NaN              Gliomas   
..                         ...                 ...                  ...   
95                    TCGA-GBM                 NaN              Gliomas   
96                    TCGA-LGG                 NaN              Gliomas   
97                    TCGA-GBM                 NaN              Gliomas   
98                    TCGA-GBM                 NaN              Gliomas   
99                    TCGA-GBM                 NaN              Gliomas   

   anatomical_site  source_material_type specimen_type  \
0             None  Blood Derived Normal       portion   
1             None         Primary Tumor       portion   
2             None         Primary Tumor       aliquot   
3             None         Primary Tumor         slide   
4             None         Primary Tumor       analyte   
..             ...                   ...           ...   
95            None         Primary Tumor       aliquot   
96            None  Blood Derived Normal       aliquot   
97            None  Blood Derived Normal       portion   
98            None  Blood Derived Normal       portion   
99            None         Primary Tumor        sample   

                   derived_from_specimen derived_from_subject    subject_id  \
0   3407f4e4-02ef-49aa-bea0-f7c496ad7371         TCGA-S9-A7QW  TCGA-S9-A7QW   
1   ae23f336-e901-4e45-b139-5e105060d3af         TCGA-HT-7609  TCGA-HT-7609   
2   183243f7-1294-490c-a5b8-af6bd8740e01         TCGA-HW-A5KL  TCGA-HW-A5KL   
3   b44d1202-4b05-545c-8f00-fdad812c28ef         TCGA-HT-7689  TCGA-HT-7689   
4   0efe9cfc-89f2-4096-8c3e-c300a8efa1ee         TCGA-02-2486  TCGA-02-2486   
..                                   ...                  ...           ...   
95  0ed7f9eb-ac18-45a7-8b0c-f4912d2d1e2d         TCGA-32-1970  TCGA-32-1970   
96  05b422cc-7cc1-4691-a337-ed5f751b7c10         TCGA-DU-8167  TCGA-DU-8167   
97  aa3459f6-b50c-4205-8e73-d07fc50ec0ac         TCGA-27-2526  TCGA-27-2526   
98  ea567da7-cbca-4657-9762-918c513c768c         TCGA-19-1791  TCGA-19-1791   
99                      initial specimen         TCGA-19-4065  TCGA-19-4065   

                      researchsubject_id  
0   765588aa-075f-477f-9c18-099d018d3a1b  
1   bfbc0d1e-2a0b-4c60-8a37-91e3ce4b79ee  
2   0639b753-4099-4ca4-adb9-763069a614ef  
3   d58b535a-6c95-43bb-ad88-0600e6447537  
4   79a68b32-29a3-47bf-9b2b-643ec7323d73  
..        


---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Specimen Field Definitions</h3>

<i><i>Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</i>
 A given specimen will have only a single subject ID and a single researchsubject ID</i>
    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>associated_project:</b> The Project associated with the specimen.</li>
  <li><b>days_to_collection:</b> The number of days from the index date to either the date a sample was collected for a specific study or project, or the date a patient underwent a procedure (e.g. surgical resection) yielding a sample that was eventually used for research.</li>
  <li><b>primary_disease_type:</b> The text term used to describe the type of malignant disease, as categorized by the World Health Organization's (WHO) International Classification of Diseases for Oncology (ICD-O). This attribute represents the disease that qualified the subject for inclusion on the ResearchProject.</li>
  <li><b>anatomical_site:</b> Per GDC Dictionary, the text term that represents the name of the primary disease site of the submitted tumor sample; recommend dropping tumor; biospecimen_anatomic_site.</li>
  <li><b>source_material_type:</b> The general kind of material from which the specimen was derived, indicating the physical nature of the source material.</li>
  <li><b>specimen_type:</b> The high-level type of the specimen, based on its how it has been derived from the original extracted sample. One of: analyte, aliquot, portion, sample, or slide.</li>
  <li><b>derived_from_specimen:</b> A source/parent specimen from which this one was directly derived.</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---


### file

The file endpoint returns information about files that meet our search criteria, and that belong to subjects, researchsubjects or specimens. You can think of these three endpoints as being nested: 

`subject.file` is the broadest search. It will give you all files that have the parameters of your query
`researchsubject.file` will give you all the files that match your query, and that are attached to specific studies and/or specimens. It will not return subject level files.
`specimen.file` will return all the files directly attached to specimen, but not those attached to the researchsubject or subject those specimens came from.


In [15]:
myquery.subject.file.run()

Getting results from database

Total execution time: 0
                        min 4.57 sec 4570 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 4969686
            More pages: True
            

In [16]:
myquery.researchsubject.file.run()

Getting results from database

Total execution time: 0
                        min 4.344 sec 4344 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 4969666
            More pages: True
            

In [17]:
myquery.specimen.file.run()

Getting results from database

Total execution time: 0
                        min 4.393 sec 4393 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 771260
            More pages: True
            

As you might expect, searching file gives us a huge number of results. This is great if you are surveying what kind of data is available, but is less useful for getting a coherent cohort. 

A better way to get files for a specific cohort is to build a more complex query using <a href="../Operators">Operators</a>. 

Another useful way to look at high level information is to use our counts feature which returns summary information rather than the full search results. Check out the <a href="../DataSummaries">Data Summaries tutorial</a> to try it.



---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>File Field Definitions</h3>

<i>A file is an information-bearing electronic object that contains a physical embodiment of some information using a particular character encoding.</i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>label:</b> Short name or abbreviation for dataset. Maps to rdfs:label.</li>
  <li><b>data_catagory:</b> Broad categorization of the contents of the data file.</li>
  <li><b>data_type:</b> Specific content type of the data file.</li>
  <li><b>file_format:</b> Format of the data files.</li>
  <li><b>associated_project:</b> A reference to the Project(s) of which this ResearchSubject is a member. The associated_project may be embedded using the ref definition or may be a reference to the id for the Project - or a URI expressed as a string to an existing entity.</li>
  <li><b>drs_uri:</b> A string of characters used to identify a resource on the Data Repo Service(DRS). Can be used to retrieve this specific file from a server.</li>
  <li><b>byte_size:</b> Size of the file in bytes. Maps to dcat:byteSize.</li>
  <li><b>checksum:</b> The md5 value for the file. A digit representing the sum of the correct digits in a piece of stored or transmitted digital data, against which later comparisons can be made to detect errors in the data.</li>
  <li><b>data_modality:</b> Data modality describes the biological nature of the information gathered as the result of an activity, independent of the technology or methods used to produce the information. Always one of "Genomic", "Proteomic", or "Imaging".</li>
  <li><b>imaging_modality:</b> An imaging modality describes the imaging equipment and/or method used to acquire certain structural or functional information about the body. These include but are not limited to computed tomography (CT) and magnetic resonance imaging (MRI). Taken from the DICOM standard.</li>
  <li><b>dbgap_accession_number:</b> The dbgap accession number for the project.</li>
</ul>  

</div>
    
---

### mutation

The file endpoint returns molecular data about specific mutations that meet our search criteria. Search is currently limited to the TCGA-READ project from GDC:

In [18]:
myquery.mutation.run()

Getting results from database

Total execution time: 0
                        min 17.445 sec 17445 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 904
            More pages: True
            

In [19]:
mutationresults = myquery.mutation.run()
mutationresults.to_dataframe()

Getting results from database

Total execution time: 0
                        min 4.694 sec 4694 ms

project_short_name  case_barcode sample_barcode_tumor  \
0            TCGA-GBM  TCGA-06-0209     TCGA-06-0209-01A   
1            TCGA-GBM  TCGA-06-5408     TCGA-06-5408-01A   
2            TCGA-GBM  TCGA-06-A5U0     TCGA-06-A5U0-01A   
3            TCGA-GBM  TCGA-12-0618     TCGA-12-0618-01A   
4            TCGA-GBM  TCGA-14-0865     TCGA-14-0865-01B   
..                ...           ...                  ...   
95           TCGA-LGG  TCGA-HT-7481     TCGA-HT-7481-01A   
96           TCGA-LGG  TCGA-HT-7857     TCGA-HT-7857-01A   
97           TCGA-LGG  TCGA-HT-7902     TCGA-HT-7902-01A   
98           TCGA-LGG  TCGA-HT-A61A     TCGA-HT-A61A-01A   
99           TCGA-LGG  TCGA-VW-A8FI     TCGA-VW-A8FI-01A   

   sample_barcode_normal         aliquot_barcode_tumor  \
0       TCGA-06-0209-10A  TCGA-06-0209-01A-01D-1491-08   
1       TCGA-06-5408-10A  TCGA-06-5408-01A-01D-1696-08   
2       TCGA-06-A5U0-10A  TCGA-06-A5U0-01A-11D-A33T-08   
3       TCGA-12-0618-10A  TCGA-12-0618-01A-01D-1492-08   
4       TCGA-14-0865-10A  TCGA-14-0865-01B-01W-0643-08   
..                   ...                           ...   
95      TCGA-HT-7481-10A  TCGA-HT-7481-01A-11D-2024-08   
96      TCGA-HT-7857-10A  TCGA-HT-7857-01A-11D-2395-08   
97      TCGA-HT-7902-10A  TCGA-HT-7902-01A-12D-2395-08   
98      TCGA-HT-A61A-10A  TCGA-HT-A61A-01A-11D-A29Q-08   
99      TCGA-VW-A8FI-10A  TCGA-VW-A8FI-01A-11D-A36O-08   

          aliquot_barcode_normal Hugo_Symbol  Entrez_Gene_Id Center  \
0   TCGA-06-0209-10A-01D-1491-08      ZNF131            7690     BI   
1   TCGA-06-5408-10A-01D-1696-08     SLC44A5          204962     BI   
2   TCGA-06-A5U0-10A-01D-A33W-08       SYCP2           10388     BI   
3   TCGA-12-0618-10A-01D-1492-08      SAMD15          161394     BI   
4   TCGA-14-0865-10A-01W-0643-08   SPATA31C2          645961     BI   
..                           ...         ...             ...    ...   
95  TCGA-HT-7481-10A-01D-2024-08      NUP155            9631     BI   
96  TCGA-HT-7857-10A-01D-2396-08       NPAP1           23742     BI   
97  TCGA-HT-7902-10A-01D-2396-08     CCDC158          339965     BI   
98  TCGA-HT-A61A-10A-01D-A29Q-08         GHR            2690     BI   
99  TCGA-VW-A8FI-10A-01D-A367-08    C16orf91          283951     BI   

   NCBI_Build  ...                        tumor_bam_uuid  \
0      GRCh38  ...  9a2d0a74-d3b7-446c-9904-4ef6810c6fed   
1      GRCh38  ...  0a42311c-ead0-448b-a329-5b2a7539b3c7   
2      GRCh38  ...  9af26d5b-ed91-4e42-b278-4eb0616926fe   
3      GRCh38  ...  5ef7f28e-1190-4778-90b8-fe1bf9c85b41   
4      GRCh38  ...  18f3d928-da55-4003-bf24-f914fdbfe96e   
..        ...  ...                                   ...   
95     GRCh38  ...  457172b0-ad6a-4786-a38a-f83d6dc5762f   
96     GRCh38  ...  69a63b3b-adb4-4d56-aee3-e8693ffef36f   
97     GRCh38  ...  9da36db6-2113-49ac-99c9-33cd31c16b2f   
98     GRCh38  ...  1a3a1718-df26-46e7-9192-8b60913974f5   
99     GRCh38  ...  730d3895-7113-4cb0-a20b-86c0b8926709   

                         normal_bam_uuid  \
0   bca04fbe-c3e7-4057-819d-aad2e5616cf4   
1   2890bafd-4249-47db-b648-a84707c7a5f0   
2   08e280f7-6fea-40c5-a232-252a520522a9   
3   bbce5166-8164-4abc-9fc7-93da532b77dd   
4   7e1a5624-8fe9-4e3e-9b57-6d6788f78931   
..                                   ...   
95  dff50cb3-435b-4a22-9136-5f04120359f5   
96  4592d83e-f782-48b9-b0b1-5c38dc3c854a   
97  5f6f32a5-36b5-4cd2-bba5-cbeb030ee9f2   
98  d55237ca-3b75-442a-bb90-7add2e7e9e8d   
99  33dd54de-b30b-461e-bc72-6d3afb39bcb4   

                                 case_id      GDC_FILTER       COSMIC  \
0   a795a841-9fd6-4eac-b871-a8c70248700f            None         None   
1   af055b98-be6a-4012-ac64-f1b6539b23d0            None         None   
2   603d2b61-b636-4054-ac95-ab66dabb3b48  common_in_exac  COSM4850882   
3   f7ba48c8-46cf-40a9-8826-7e1f67d57171            None         None   
4   7d5c3699-ce28-4129-8391-736f670e8beb        wga_pair         None   
..                                   ...    